In [ ]:
'''
Train a Simple Neural Network of Mnist Dataset
'''

In [ ]:
# import libraries with TensorFlow fallback
import numpy as np
import matplotlib.pyplot as plt
try:
    import seaborn as sns
except Exception as _e:
    sns = None
    print('seaborn import failed:', _e)

TF_AVAILABLE = False
try:
    import tensorflow as tf
    from tensorflow.keras.datasets import mnist
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Flatten
    from tensorflow.keras.utils import to_categorical
    TF_AVAILABLE = True
    print("Using TensorFlow:", tf.__version__)
except Exception as e:
    print("TensorFlow unavailable or failed to import:", e)
    from sklearn.datasets import fetch_openml
    from sklearn.model_selection import train_test_split
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
if TF_AVAILABLE:
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    y_train = to_categorical(y_train, num_classes=10)
    y_test = to_categorical(y_test, num_classes=10)
    print("MNIST loaded via TensorFlow.")
else:
    print("Fetching MNIST via scikit-learn (fetch_openml).")
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
    X = X.astype('float32') / 255.0
    y = y.astype(int)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("MNIST loaded via fetch_openml. Shapes:", X_train.shape, X_test.shape)


In [ ]:
# Train a model depending on availability
if TF_AVAILABLE:
    model = Sequential([
        Flatten(input_shape=(28,28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=2, batch_size=128, validation_split=0.1)
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test accuracy: {acc:.4f}')
else:
    # scale and train a scikit-learn MLPClassifier (faster to get a working notebook)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=20, verbose=True)
    mlp.fit(X_train_scaled, y_train)

    y_pred = mlp.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc:.4f}')
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion matrix (MLPClassifier)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()
